<a href="https://colab.research.google.com/github/usmanyousaaf/computer-vision-projects/blob/main/K_Mean_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from PIL import Image
import os
import matplotlib.pyplot as plt

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/datasetz.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

### Set the path to the director   containing the images

In [ ]:
train_dir = "/content/datasetz/traning"
test_dir = "/content/datasetz/test"


### Get a list of all the image file names in the directory

In [ ]:
# Get a list of all the image file names in the training directory
train_img_files = os.listdir(train_dir)

# Initialize an empty array to store the training image data
train_X = np.empty((0,3), dtype=np.uint8)

### Loop through all the images in the training directory

In [ ]:

for img_file in train_img_files:
    # Load the image and convert it to a numpy array
    img = np.array(Image.open(os.path.join(train_dir, img_file)))

    # Reshape the array so that each pixel is represented by a single row
    rows, cols, bands = img.shape
    img_data = img.reshape(rows*cols, bands)

    # Append the image data to the training X array
    train_X = np.vstack((train_X, img_data))

### Choose the number of clusters

In [ ]:
k = 3

### Initialize k centroids randomly

In [ ]:
kmeans = KMeans(n_clusters=k, init='random').fit(train_X)

### Repeat the following steps until convergence

In [ ]:

while True:
    # Assign each pixel to the nearest centroid
    train_labels = kmeans.predict(train_X)

    # Update the position of each centroid to the mean of the pixels assigned to it
    old_centroids = kmeans.cluster_centers_.copy()
    for i in range(k):
        kmeans.cluster_centers_[i] = np.mean(train_X[train_labels == i], axis=0)

    # Check for convergence
    if np.allclose(kmeans.cluster_centers_, old_centroids):
        break

### Loop through all the images in the testing directory

In [ ]:

test_img_files = os.listdir(test_dir)
for img_file in test_img_files:
    # Load the image and convert it to a numpy array
    img = np.array(Image.open(os.path.join(test_dir, img_file)))

    # Reshape the array so that each pixel is represented by a single row
    rows, cols, bands = img.shape
    img_data = img.reshape(rows*cols, bands)

    # Assign each pixel to the nearest centroid
    test_labels = kmeans.predict(img_data)

    # Reshape the cluster assignments back into an image for visualization
    img_seg = test_labels.reshape(rows, cols)

    # Plot the original image and the segmented image side by side
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(img)
    axs[0].set_title("Original Image")
    axs[1].imshow(img_seg)
    axs[1].set_title("Segmented Image")
    plt.show()